In [1]:
#************************* MAIN SCRIPT **********************************
import sys
import math
import h5py
import matplotlib.pyplot as plt
import os
import math
from scipy import integrate
import numpy as np
import csv
#os.path.append('/Users/ajc/Core/Projects/ATS-Data/OR-CONDO/PyScript')
#import readdata

import datetime as DT
import matplotlib.dates as mdates
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}
matplotlib.rc('font', **font)

In [2]:
time_origin = DT.datetime(2012,1,1,0,0,0)

In [3]:
#path ='/Users/ajc/FUSE/simulations/forcing_data/Barrow-Validation/Barrow-2010_2016-updated-smooth.h5'
path ='/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/Barrow_Alaska/Barrow-2010_2016-updated-smooth-30pc.h5'


#path_cesm ='/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8.5-2006-2100_dm1985-2018.h5' #old projection data with rain trend error
#path_cesm ='/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM.h5' # data with my script and single linear trend

path_cesm ='/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-ET.h5' # data with Ethan script and double linear trend


path_wl_full = '/Users/ajc/Core/PreProcessing/barrow_icewedges/hydro_connectivity/waterlevel-3D-wellC37-2012-2014.h5'



In [4]:
forcing = h5py.File(path,'r')
forcing_cesm = h5py.File(path_cesm,'r')
observed_path = h5py.File(path_wl_full,'r')


In [5]:
r = int(len(forcing['precipitation rain [m s^-1]'])%365.)
def reshape_forcing(forcing_data,nyear=-1,s=0,e=-1):
    rain1 = [r*86400*10 for r in forcing_data['precipitation rain [m s^-1]'][s:e]]
    L = len(rain1)%nyear
    if L != 0:
        rain1.append(rain1[-1])
    rain = np.reshape(rain1, (nyear,365))
    rain_time = [r/86400. - 2*365*0 for r in forcing_data['time [s]'][s:e]]

    temp1 = [r for r in forcing_data['air temperature [K]'][s:e]]
    if L != 0:
        temp1.append(temp1[-1])
    temp = np.reshape(temp1, (nyear,365))
    temp_time = [r/86400. for r in forcing_data['time [s]'][s:e]]
    if L != 0:
        rain_time.append(rain_time[-1]+86400)
    rain_time = [2010 + t/365. for t in rain_time] # ADDED today
    time = np.reshape(rain_time, (nyear,365))

    return rain, time, temp
#------------------- 2014 ----------------------------
observed_wt = dict()

observed_wt['watertable'] = np.array([ (w-101325)/(1000*9.8) for w in observed_path['ponded depth C37 [m]), left']])
observed_wt['time'] = np.array([t/86400 - 2*365*0 for t in observed_path['time [s]']])


In [6]:
s = 2*365
e = s + 3*365

Prain, Time, Temp = reshape_forcing(forcing_data=forcing,nyear=3,s=s,e=e)
Prain_cesm, Time_cesm, Temp_cesm = reshape_forcing(forcing_data=forcing_cesm,nyear=95)

In [7]:
def consecutive(datad, stepsize=1):
    d = np.split(datad, np.where(np.diff(datad) != stepsize)[0]+1)
    #l = np.argmax(np.array([len(x) for x in d]))
    L = np.array([len(x) for x in d])
    d2 = []
    for i, l in enumerate(L):
        if l > 7:
            d2.append(d[i])
    flat_d2 = [item for sublist in d2 for item in sublist]
    return flat_d2

def get_time_format(dat):
    Dat = []
    year = int(dat[0])
    year_start = DT.date(year,1,1)
    dateFormatted = year_start.strftime("%y")
    #year_start = format(dateFormatted)
    for i in dat:
        f = float(i)
        delta = year_start + DT.timedelta(days = int((i-year)*365) )
        #delta = format(dateFormatted) + DT.timedelta(days = int((i-year)*365) )
        Dat.append(delta)
    print ('Time formate:', len(dat), len(Dat), Dat[0], year_start)
    return Dat

In [ ]:
def get_water_level(Prain_,Time_,Temp_):
    color = ['k','r','g','m','c','y']
    fitted_wt = [] #dict()
    fitted_Time = []
    nyear = len(Prain_)
    for i in range(nyear):
        
        temp_interval = consecutive(np.where(Temp_[i] > 273.15)[0])
        I0, In = temp_interval[0]-1,temp_interval[-1]+1
        
        Rain = Prain_[i][I0:In]*100
        
        watertable = []
        if i >-1:
            watertable.append(0.1)
            fitted_Time.append(Time_[i][I0-1:In+1])
            fitted_Time = [0 + t for t in fitted_Time]
        else:
            fitted_Time.append(Time_[i][I0:In+1])
        Rain_interval = np.where(Rain > 2.5)[0]
        print (Rain_interval)
        #print ('WL:', i, watertable[-1])
        #fitted_Time.append(0)
        s = 0.2 - 0.01*(Rain_interval[1] - Rain_interval[0])/7 # 0.01 m per week decrease
        print ('S: ',s)
        for j in range(Rain_interval[0]):
            drain1 = s - 0.008*j

            watertable.append(drain1)
        print ('WL1" ',i, watertable)
        for j, val in enumerate(Rain_interval):
            r1 = watertable[-1] + (Rain[val]/1000)**0.65
            if watertable[-1] >0.1 and watertable[-1] <0.2:#heavy rain
                r1 = watertable[-1] + (Rain[val]/1000)**1.1
            if (val<Rain_interval[-1]):
                dI = Rain_interval[j+1] - Rain_interval[j]
            else:
                dI = len(Rain) - (Rain_interval[-1])
            water_2 = []
            for k in range(dI):
                if j < 3:
                    drain = r1 - k*0.005
                else:
                    drain = r1 - k*0.0025
                if drain > 0 :
                    water_2.append(drain)
                else:
                    water_2.append(0.0)

            for d1 in water_2:
                watertable.append(d1)
        watertable.append(0.1)
        fitted_wt.append(watertable)
        
        
        
    #print ('Length: ', len(fitted_Time), len(fitted_wt),fitted_Time[0])
    return np.array(fitted_Time), np.array(fitted_wt)

In [ ]:
#fitted_Time,fitted_wt = get_water_level(Prain_=Prain,Time_=Time,Temp_=Temp)

In [8]:
def get_water_level_Modified(Prain_,Time_,Temp_):
    color = ['k','r','g','m','c','y']
    fitted_wt = [] #dict()
    fitted_Time = []
    nyear = len(Prain_)
    for i in range(nyear):
        
        temp_interval = consecutive(np.where(Temp_[i] > 273.15)[0])
        I0, In = temp_interval[0]-1,temp_interval[-1]+1
        
        temp_I_orig = np.where(Temp_[i] > 273.15)[0]
        #print (temp_I_orig[0], I0)
        Rain = Prain_[i][I0:In]*100
        
        watertable = []
        if i >-1:
            watertable.append(0.1)
            fitted_Time.append(Time_[i][I0-1:In+1])
            fitted_Time = [0 + t for t in fitted_Time]
        else:
            fitted_Time.append(Time_[i][I0:In+1])
        Rain_interval = np.where(Rain > 2.5)[0]
        if (I0 - temp_I_orig[0] > 0):
            s = 0.2 - 0.001*(I0 - temp_I_orig[0]) # 1 mm per day drainage
        else:
            s = 0.2
        print ('S: ', s, I0, temp_I_orig[0])
        for j in range(Rain_interval[0]):
            drain1 = s - 0.005*j
            watertable.append(drain1)

        for j, val in enumerate(Rain_interval):
            #print ('A:', watertable[-1])
            if (val<Rain_interval[-1]):
                dI = Rain_interval[j+1] - Rain_interval[j]
            else:
                dI = len(Rain) - (Rain_interval[-1])
            #print ('F: ',j,Rain_interval,dI)
            water_2 = []
            if watertable[-1] <=0.1:
                r1 = watertable[-1] + (Rain[val]/1000)**0.75
                #print (1./dI*0.001)
                for k in range(dI):
                    rate = 0.002 
                    drain = r1 - k*rate
                    if drain > 0 :
                        water_2.append(drain)
                    else:
                        water_2.append(0.0)
            elif watertable[-1] >0.1 and watertable[-1] <=0.2:# high water table more run-off and less run-on
                r1 = watertable[-1] + (Rain[val]/1000)**4.0
                rate = 0.005 
                #print ('R: ',rate)
                for k in range(dI):
                    drain = r1 - k*rate
                    if drain > 0 :
                        water_2.append(drain)
                    else:
                        water_2.append(0.0)
            else:
                r1 = 0.2 + (Rain[val]/1000)**4.0
                r1 = watertable[-1] + (Rain[val]/1000)**4.0
                rate = 0.005
                for k in range(dI):
                    drain = r1 - k*rate
                    if drain > 0 :
                        water_2.append(drain)
                    else:
                        water_2.append(0.0)
                        
                    
                print ('Water Table greater than 0.2',watertable[-1])


            for d1 in water_2:
                watertable.append(d1)
        watertable.append(0.1)
        fitted_wt.append(watertable)
        
        #print (len(np.concatenate(fitted_Time)),len(np.concatenate(fitted_wt)))
        
    print ('Length: ', len(fitted_Time), len(fitted_wt))#,fitted_Time[0])
    return np.array(fitted_Time), np.array(fitted_wt)

In [9]:
fitted_Time,fitted_wt = get_water_level_Modified(Prain_=Prain,Time_=Time,Temp_=Temp)

S:  0.182 157 139
S:  0.183 159 142
S:  0.15600000000000003 166 122
Length:  3 3


In [10]:
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/'

In [11]:
%matplotlib qt
fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
ax = axs.ravel()

time_ob = [2010 + t/365. for t in observed_wt['time']]
time_ob = get_time_format(time_ob)
Z_t = 4.71264
#print (time_ob[0])

Time_rainf = get_time_format(np.concatenate(Time))

time_tt = []
for i in range(0,3):
    #ax = plt.subplot(311)
    xfmt = mdates.DateFormatter('%m/%y')
    ax[i].xaxis.set_major_formatter(xfmt)

    ob_lower = [ x - 0.03 if x != 0.1 else 0.1 for x in observed_wt['watertable']]
    ob_upper = [ x + 0.03 if x != 0.1 else 0.1 for x in observed_wt['watertable']]
    ob_lower = [x + Z_t for x in ob_lower]
    ob_upper = [x + Z_t for x in ob_upper]
    
    if (i==2):
        ax[i].plot(time_ob[1:],observed_wt['watertable'][1:]+Z_t,'r',label='Observed')
    else:
        ax[i].plot(time_ob[1:],observed_wt['watertable'][1:]+Z_t,'r')
    ax[i].fill_between(time_ob[1:], ob_lower[1:], ob_upper[1:], color='lightgray')

    Time_f = get_time_format(fitted_Time[i])
    print (Time_f[0],Time_f[-1], time_ob[0])
    
    d = [x + Z_t for x in fitted_wt[i]]
    d = [x1 if x1 >Z_t else Z_t for x1 in d]
    if i ==2:
        ax[i].plot(Time_f,d,color='k',label='Modeled')
        ax[i].axhline(y=Z_t,linestyle='--',color='g',label='Trough elevation')
    else:
        ax[i].plot(Time_f,d,color='k')
        ax[i].axhline(y=Z_t,linestyle='--',color='g')
    
    ax[i].set_xticks(Time_f[::31])
    ax[i].set_xlim((Time_f[0], Time_f[-1]))

    if (i==2):
        ax[i].set_xlabel('Time [m/y]',fontsize=12, fontweight='normal')

    ax[i].set_ylim(4.4,5.1)
    ax[i].set_ylabel('Water level [m]',fontsize=12, fontweight='normal')
    


    ax1 = ax[i].twinx()
    ax1.xaxis.set_major_formatter(xfmt)
    ax1.plot(Time_rainf,-100*np.concatenate(Prain),'b')
   
    ax1.set_ylabel('Precipitation Rain [mm]',color='b')
    ax1.set_ylim([-50.0, 0.5])
    ax1.set_yticklabels(np.round(np.linspace(50,0,6,dtype='i'),2))
    ax1.set_xticks(Time_f[::31])

    ax1.set_xlim((Time_f[0], Time_f[-1]))
    
    #ax1 = ax[i].twinx()
    #ax1.xaxis.set_major_formatter(xfmt)
    #ax1.plot(Time_rainf,-100*np.concatenate(Prain),'b')
    #ax1.set_xticks(Time_rainf[::31])
    #ax1.set_xlim((Time_rainf[0], Time_rainf[-1]))
    #ax1.set_ylabel('Precipitation Rain [mm]',color='b',fontsize=12, fontweight='normal')
    #ax1.set_ylim([-50, 0.5])
    #ax1.set_yticklabels(np.round(np.linspace(50,0,6,dtype='i'),2))
ax[2].legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.9,-0.32, .1, .1))
plt.tight_layout()
print (outfile)
#plt.savefig(outfile+'watertable_model-modified.pdf', bbox_inches='tight', dpi=400)


findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


Time formate: 313 313 2012-06-08 2012-01-01
Time formate: 1095 1095 2012-01-01 2012-01-01
Time formate: 115 115 2012-06-05 2012-01-01
2012-06-05 2012-09-27 2012-06-08
Time formate: 101 101 2013-06-08 2013-01-01
2013-06-08 2013-09-16 2012-06-08
Time formate: 100 100 2014-06-15 2014-01-01
2014-06-15 2014-09-22 2012-06-08
/Users/ajc/Documents/MyPapers/TroughBC/figures/


In [12]:
fitted_Time_m,fitted_wt_m = get_water_level_Modified(Prain_=Prain_cesm,Time_=Time_cesm,Temp_=Temp_cesm)

S:  0.187 156 143
S:  0.2 153 154
S:  0.19 153 143
S:  0.167 150 117
S:  0.192 155 147
S:  0.17700000000000002 162 139
S:  0.188 149 137
S:  0.184 157 141
S:  0.169 152 121
S:  0.195 135 130
S:  0.2 136 137
S:  0.194 154 148
S:  0.2 146 147
S:  0.2 138 139
S:  0.125 145 70
S:  0.2 128 129
S:  0.192 132 124
S:  0.195 136 131
S:  0.191 144 135
S:  0.185 153 138
S:  0.17200000000000001 143 115
S:  0.2 126 127
Water Table greater than 0.2 0.2000000004011011
S:  0.18100000000000002 153 134
S:  0.193 141 134
S:  0.186 145 131
S:  0.2 148 149
S:  0.2 148 149
S:  0.192 135 127
S:  0.17 146 116
S:  0.196 135 131
S:  0.195 143 138
S:  0.2 133 134
S:  0.2 144 145
S:  0.196 139 135
S:  0.18100000000000002 135 116
S:  0.2 138 139
S:  0.2 134 135
S:  0.2 133 134
S:  0.189 138 127
S:  0.192 128 120
S:  0.2 127 128
S:  0.191 135 126
S:  0.2 122 123
S:  0.191 136 127
S:  0.17600000000000002 131 107
S:  0.097 137 34
S:  0.17500000000000002 127 102
S:  0.2 121 122
S:  0.2 129 130
S:  0.2 126 127
S:  0.18

In [13]:
for i in range(0,1,1):
    plt.plot(fitted_Time_m[i],fitted_wt_m[i],'k.')
print (len(fitted_wt_m))
print (fitted_wt_m.shape)
fitted_wt_m = np.concatenate(fitted_wt_m)

fitted_Time_m = np.concatenate(fitted_Time_m)
plt.plot(fitted_Time_m[:700],fitted_wt_m[:700],'r')
print (len(fitted_wt_m), len(fitted_Time_m))
#print (fitted_Time_m[0])


95
(95,)
14893 14893


In [14]:
plt.plot(fitted_Time_m,fitted_wt_m)

In [15]:
#print (fitted_Time_m[:2])
# THIS TIME IS CONSISTENT WITH THE CESM start time, start time is half day
Time_corrected = np.array([np.round((x - 2010)*86400*365,2) for x in fitted_Time_m]) 
#print (t0)

#Time = np.array([t for t in range()])
#Time = forcing_cesm['time [s]']


In [25]:
print (Time_corrected[0]/86400, 1.2312E7/86400.)


155.5 142.5


In [20]:
path_wl_cesm = h5py.File('/Users/ajc/Core/PreProcessing/barrow_icewedges/hydro_connectivity/waterlevel-cesm-2006-2100_updated-ET.h5','w')
fitted_wt_pres =  np.array([(f+0*0.0025)*1000*9.8 + 101325. for f in fitted_wt_m])
fitted_wt_pres = np.where(fitted_wt_pres > 101325, fitted_wt_pres, 101325)
fitted_wt_pres = np.array([np.round(f,2) for f in fitted_wt_pres])
#fitted_Time_s =  np.array([f*86400 for f in fitted_Time_m]) # used in old simulations
#fitted_Time_s = Time #Time_corrected ## commented on Feb 9 2021
fitted_Time_s = Time_corrected
path_wl_cesm.create_dataset('pressure [Pa]',data=fitted_wt_pres)
path_wl_cesm.create_dataset('time [s]',data=fitted_Time_s)
path_wl_cesm.close()

In [27]:
path_wl_cesm = h5py.File('/Users/ajc/Core/PreProcessing/barrow_icewedges/hydro_connectivity/waterlevel-cesm-2006-2100_updated_corrected.h5','r')
path_wl_cesm2 = h5py.File('/Users/ajc/Core/PreProcessing/barrow_icewedges/hydro_connectivity/waterlevel-cesm-2006-2100_updated.h5','r')
path_wl_cesm_et = h5py.File('/Users/ajc/Core/PreProcessing/barrow_icewedges/hydro_connectivity/waterlevel-cesm-2006-2100_updated-ET.h5','r')

In [30]:
d = [ (p1-101325)/(9.8*1000) for p1 in path_wl_cesm['pressure [Pa]']]
t = [t1/86400 for t1 in path_wl_cesm['time [s]']]

#d2 = [ (p1-101325)/(9.8*1000) for p1 in path_wl_cesm2['pressure [Pa]']]
#t2 = [t1/86400 for t1 in path_wl_cesm2['time [s]']]

d2 = [ (p1-101325)/(9.8*1000) for p1 in path_wl_cesm_et['pressure [Pa]']]
t2 = [t1/86400 for t1 in path_wl_cesm_et['time [s]']]

plt.plot(t,d,'k')
plt.plot(t2,d2,'r')
#plt.xlim(0,300)
print (t2[0],t2[-1])

155.5 34618.5


In [ ]:
# projected water table 
cesm_data = h5py.File('/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8_5-2006-2100_snowadjusted_supersmoothDM_2006.h5','r')

In [ ]:
cesm_rain = np.reshape(cesm_data['precipitation rain [m s^-1]'],(-1,365))
cesm_temp = np.reshape(cesm_data['air temperature [K]'],(-1,365))

barrow_rain2013 = rain[1]*100
barrow_rain2014 = rain[2]*100
#[r*86400*1000 for r in forcing['precipitation rain [m s^-1]'][365*3:365*4]]

In [ ]:
plt.plot(cesm_rain[6]*1000*86400)
plt.plot(barrow_rain2014,'r.')

In [ ]:
plt.plot(Temp,C)
plt.xlabel('Soil temperature ')
plt.ylabel('Decomposition rate')